In [ ]:
import healpy as hp
import mapsims
from glob import glob
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
sim = mapsims.from_config(["common.toml", "noise.toml"], override={"channels":"tube:ST0"})

noise = sim.other_components["noise"]

In [ ]:
onthefly = sim.execute()

In [ ]:
onthefly.keys()

In [ ]:
hitmaps = {}
sky_fractions = {}
for i_tube in range(4):
    tube = f"ST{i_tube}"
    chs = mapsims.parse_channels("tube:"+tube)[0]
    hitmaps[tube], sky_fractions[tube] = noise.get_hitmaps(tube)
    hitmaps[tube] = hp.ud_grade(hitmaps[tube],
            mapsims.runner.get_default_so_resolution(chs))
    
    for ch, h in zip(chs, hitmaps[tube]):
        h /= h.max()
        #h[h<.01] = 0
        #hp.mollview(h, title=ch.tag)

In [ ]:
all_cl = {}
for i_tube in [0]:
    tube = f"ST{i_tube}"
    ellmax = int(1e4)
    m = {}
    cl = []
    folder = "output/noise/0000/"
    for i, ch in enumerate(mapsims.parse_channels("tube:"+tube)[0]):
        filename = glob(folder + f"*{ch.tag}*1_of_1*")[0]
        print("reading " + filename)
        m[i] = onthefly[ch.tag]
        m[i][m[i] == hp.UNSEEN] = 0
        
        npix = len(m[i][0])
        sky_fraction_before=hp.mask_good(m[i][1]).sum()/npix

        m[i] *= hitmaps[tube][i]
        
        hp.mollview(m[i][1], title=ch.tag)
        nside = hp.npix2nside(npix)
        sky_fraction=hp.mask_good(m[i][1]).sum()/npix
        print("Sky fraction loss due to additional masking {:.1%} {:.1%} {:.1%}".format(
            sky_fraction_before, sky_fraction, sky_fraction_before - sky_fraction))
        cl.append(np.array(hp.anafast(m[i], lmax=min(3*nside-1,ellmax), use_pixel_weights=True)) / np.mean(hitmaps[tube][i]**2))
    #cl.append(np.array(hp.anafast(m[0],m[1], lmax=min(3*nside-1,ellmax), use_pixel_weights=True)) / np.mean(hitmaps[tube][i]**2))
    cl = np.array(cl)
    all_cl[i_tube] = cl
cl = all_cl

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mapsims

In [ ]:
from glob import glob

In [ ]:
import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {'family':'sans-serif',
                  'weight' : 'normal', 'size' : 16}

In [ ]:
import pickle

In [ ]:
%matplotlib inline

In [ ]:
ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube="ST0")

In [ ]:
#m_onthefly = hp.synfast(ps_P[0] * sky_fractions["ST0"][0], nside=1024)
m_onthefly = hp.synfast(ps_P[0], nside=1024)

In [ ]:
#m_onthefly /= np.sqrt(hitmaps["ST0"][0])

In [ ]:
#hitmaps["ST0"][0][:] = 1
npix = len(hitmaps["ST0"][0])
#hitmaps["ST0"][0][npix//2-3500000:npix//2+3500000] = 0

In [ ]:
m_onthefly[hitmaps["ST0"][0] == 0] = hp.UNSEEN

In [ ]:
hp.mollview(hitmaps["ST0"][0])

In [ ]:
hp.mollview(m_onthefly) 

In [ ]:
m_onthefly *= hitmaps["ST0"][0]

In [ ]:
fsky = np.mean((hitmaps["ST0"][0]!=0)**2)

In [ ]:
(hitmaps["ST0"][0]!=0).sum() / len(hitmaps["ST0"][0])

In [ ]:
fsky

In [ ]:
cl_onthefly = hp.anafast(m_onthefly) / np.mean(hitmaps["ST0"][0]**2)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(20/3*2, 5*2))
ylim = (1e-7, 1e-2)
i_tube = 0
for (tube, chs) in [("ST0",noise.tubes["ST0"])]:
    ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=tube)

    if tube.startswith("S"):
    
        m = {}
        for i, ch in enumerate(chs):
            ax[i_tube][i].loglog(ell, ps_P[i])
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
        for pol in [1,2]:
            for i, ch in enumerate(chs):
                len_cl = len(cl[i_tube][i])
                #ax[i_tube][i].loglog(cl[i_tube][i][pol])
                ax[i_tube][i].loglog(cl_onthefly)

                ax[i_tube][i].set_ylim(*ylim)
                ax[i_tube][i].set_title(ch, fontsize=16)
                
        i_tube += 1